In [37]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split

In [38]:
def extract_frames_from_video(video_path, frame_count=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(total_frames // frame_count, 1)

    for i in range(0, total_frames, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames

In [39]:
def load_video_dataset(data_dir, frame_count=5):
    X, y = [], []
    class_labels = {'YouTube-real': 0, 'Celeb-synthesis': 1, 'Celeb-real': 0}
    
    for label, value in class_labels.items():
        subdir = os.path.join(data_dir, label)
        for file in os.listdir(subdir):
            if file.endswith('.mp4'):
                video_path = os.path.join(subdir, file)
                frames = extract_frames_from_video(video_path, frame_count=frame_count)
                for frame in frames:
                    X.append(frame)
                    y.append(value)
    
    return X, y


In [40]:
def apply_sobel_filter(image, target_size=(256, 256)):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        sobel = cv2.magnitude(sobelx, sobely)
        sobel_resized = cv2.resize(sobel, target_size)
        return sobel_resized
    except Exception as e:
        print(f"Erro ao aplicar o filtro Sobel: {e}")
        return None

In [41]:
def process_in_batches(X, batch_size=100, target_size=(256, 256)):
    X_processed = []
    for i in range(0, len(X), batch_size):
        batch = X[i:i + batch_size]
        batch_sobel = [apply_sobel_filter(frame, target_size) for frame in batch]
        batch_sobel = [sobel for sobel in batch_sobel if sobel is not None]
        X_processed.extend(batch_sobel)
    return np.array(X_processed)


In [42]:
# Carregar o conjunto de dados de vídeos
data_dir = 'Celeb-DF'
X, y = load_video_dataset(data_dir)

In [43]:
# Processar em lotes
X_sobel = process_in_batches(X, batch_size=100, target_size=(256, 256))

# Normalizar os dados
X_sobel = X_sobel / 255.0

# Expandir as dimensões para incluir o canal de cor
X_sobel = np.expand_dims(X_sobel, axis=-1)

In [44]:
# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_sobel, y, test_size=0.2, random_state=42)

In [45]:
# Adicionar um canal de cor extra para compatibilidade com EfficientNet
X_train = np.repeat(X_train, 3, axis=-1)
X_test = np.repeat(X_test, 3, axis=-1)

In [46]:
# Pré-processar os dados com preprocess_input do EfficientNet
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [47]:
# Construir e treinar o modelo EfficientNetB0
def build_efficientnet(input_shape):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [48]:
input_shape = (X_sobel.shape[1], X_sobel.shape[2], 3)
efficientnet_model = build_efficientnet(input_shape)

In [49]:
efficientnet_model.fit(X_train, np.array(y_train), epochs=10, batch_size=32, validation_data=(X_test, np.array(y_test)))

Epoch 1/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 651s 4s/step - accuracy: 0.7655 - loss: 0.9340 - val_accuracy: 0.7743 - val_loss: 0.5503
Epoch 2/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 614s 4s/step - accuracy: 0.8390 - loss: 0.4021 - val_accuracy: 0.7585 - val_loss: 0.5026
Epoch 3/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 611s 4s/step - accuracy: 0.8706 - loss: 0.3246 - val_accuracy: 0.8522 - val_loss: 0.3458
Epoch 4/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 608s 3s/step - accuracy: 0.8577 - loss: 0.3659 - val_accuracy: 0.8709 - val_loss: 0.2917
Epoch 5/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 609s 3s/step - accuracy: 0.8816 - loss: 0.2955 - val_accuracy: 0.8724 - val_loss: 0.3227
Epoch 6/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 608s 3s/step - accuracy: 0.9214 - loss: 0.1903 - val_accuracy: 0.8565 - val_loss: 0.5334
Epoch 7/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 609s 3s/step - accuracy: 0.9346 - loss: 0.1694 - val_accuracy: 0.8335 - val_loss: 0.3899
Epoch 8/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 609s 3s/step - accuracy: 0.9471 - loss: 0.1368 - val_accu

In [50]:
efficientnet_model.save('Model/meu_modelo_Sobel_Efficientnet_Model.h5')